### **Motivation**

Getting financial statment data online is quite difficult or at least not automatic as we could think. Here, I aim at automating this process of getting finanical data from, for instance, YahooFinance!

### **Libraries**

I will use 4 libraries: 


1.   Pandas : To deal with the large amount of data
2.   urllib.request: to manage the webscrapping part. it basically get all the information on a specific web page.
3.   BeautifulSoup: To take off the web language around the data I need.
4.   Datetime: To get the current year automatically










In [58]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request as ur
from datetime import date

### **Build URL**

For instance: https://finance.yahoo.com/quote/MSFT/financials?p=MSFT

In [59]:
ticker = 'MSFT'
# URL link 

url_IncomeStmt = 'https://finance.yahoo.com/quote/%s/financials?p=%s' %(ticker,ticker)
url_BalanceSheet = 'https://finance.yahoo.com/quote/%s/balance-sheet?p=%s' %(ticker,ticker)
url_CashFlowStmt = 'https://finance.yahoo.com/quote/%s/cash-flow?p=%s' %(ticker,ticker)

### **Get the data**
Urllib wikk return all the information of the web page into a lxml format. This is important to know since you have to specify the format when using BeautifulSoup.

##### **Income Statement**

In [60]:
read_data_IS = ur.urlopen(url_IncomeStmt).read() 
raw_IS = BeautifulSoup(read_data_IS,'lxml')


In [61]:
ls= [] # Create empty list
for data in raw_IS.find_all('div'): 
  #Find all data structure that is 'div'
  ls.append(data.string) # add each element one by one to the list


In [62]:
CleanData_IS = list(filter(None,ls))

In [63]:
# fixing some issues
CleanData_IS= CleanData_IS[13:]
CleanData_IS.remove('Quarterly')
CleanData_IS.remove('Expand All')
CleanData_IS.insert(6,'Total Revenue')
CleanData_IS.insert(24,'Operating Expense')
CleanData_IS.insert(36,'Net Non-Operating Interest Income Expense')
CleanData_IS.insert(42,'Other Income Expense')
CleanData_IS.insert(60,'Net Income Common Stockholders ')


In [64]:
data_IS = list(zip(*[iter(CleanData_IS)]*6))

In [65]:
Income_stmt = pd.DataFrame(data_IS[0:])
Income_stmt.columns = Income_stmt.iloc[0] # Name columns to first row of dataframe
Income_stmt = Income_stmt.iloc[1:,] # start to read 1st row
Income_stmt = Income_stmt.T # transpose dataframe
Income_stmt.columns = Income_stmt.iloc[0] #Name columns to first row of dataframe
Income_stmt.drop(Income_stmt.index[0],inplace=True) #Drop first index row
Income_stmt.index.name = '' # Remove the index name
Income_stmt.rename(index={'ttm': '12/31/%s' %date.today().year},inplace=True) #Rename ttm in index columns to end of the year
Income_stmt.rename(columns={'Annual':''},inplace=True)

In [66]:
Income_stmt

ttm,Total Revenue,"46,674,000","100,440,000",Operating Expense,"56,149,000",Net Non-Operating Interest Income Expense,Other Income Expense,"56,474,000","8,978,000",Net Income Common Stockholders,"47,496,000",-,-,-,-,"56,149,000","90,965,000","47,496,000","45,065,440",-,"2,543,000","-2,543,000","59,017,000","71,487,000","46,674,000","12,470,000","47,496,000","2,890,000","2,890,000","68,597,000",0,"459,440"
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6/30/2020,"143,015,000","46,078,000","96,937,000","43,978,000","52,959,000","-2,591,000","2,668,000","53,036,000","8,755,000","44,281,000","44,281,000",0.0058,0.0058,"7,610,000","7,683,000","52,959,000","90,056,000","44,281,000","42,019,820","2,680,000","2,591,000","-2,591,000","55,627,000",-,"46,078,000","12,796,000","44,281,000","2,708,000","2,708,000","65,715,000",0,"446,820"
6/30/2019,"125,843,000","42,910,000","82,933,000","39,974,000","42,959,000","-2,686,000","3,415,000","43,688,000","4,448,000","39,240,000","39,240,000",0.0051,0.0051,"7,673,000","7,753,000","42,959,000","82,884,000","39,240,000","36,122,144",-,"2,686,000","-2,686,000","46,374,000",-,"42,910,000","11,682,000","39,240,000","3,472,000","3,472,000","54,584,000",0,"354,144"
6/30/2018,"110,360,000","38,353,000","72,007,000","36,949,000","35,058,000","-2,733,000","4,149,000","36,474,000","19,903,000","16,571,000","16,571,000",0.0022,0.0021,"7,700,000","7,794,000","35,058,000","75,302,000","16,571,000","13,499,160",-,"2,733,000","-2,733,000","39,207,000",-,"38,353,000","10,261,000","16,571,000","4,208,000","4,208,000","45,260,000",0,"1,136,160"
6/30/2017,"89,950,000","34,261,000","55,689,000","33,057,000","22,632,000","-2,222,000","2,739,000","23,149,000","1,945,000","21,204,000","21,204,000",0.0027,0.0027,"7,746,000","7,832,000","22,326,000","67,318,000","21,204,000","18,453,200",-,"2,222,000","-2,222,000","25,371,000",-,"34,261,000","8,778,000","21,204,000","2,990,000","2,990,000","31,159,000",0,"239,200"
"147,114,000",Cost of Revenue,Gross Profit,"44,291,000",Operating Income,"-2,543,000","2,868,000",Pretax Income,Tax Provision,"47,496,000",Diluted NI Available to Com Stockholders,Basic EPS,Diluted EPS,Basic Average Shares,Diluted Average Shares,Total Operating Income as Reported,Total Expenses,Net Income from Continuing & Discontinued Oper...,Normalized Income,Interest Income,Interest Expense,Net Interest Income,EBIT,EBITDA,Reconciled Cost of Revenue,Reconciled Depreciation,Net Income from Continuing Operation Net Minor...,Total Unusual Items Excluding Goodwill,Total Unusual Items,Normalized EBITDA,Tax Rate for Calcs,Tax Effect of Unusual Items,Learn more


##### **Balance Sheet**

In [67]:
read_data_BS = ur.urlopen(url_BalanceSheet).read() 
raw_BS = BeautifulSoup(read_data_BS,'lxml')

In [68]:
ls_BS= [] 
for data in raw_BS.find_all('div'): 
  ls_BS.append(data.string) 

In [69]:
data

<div><script type="text/javascript">window._loadEvt = false; 
window._adPerfData = []; 
window._adPosMsg = []; 
window._perfMark = function _perfMark (name) {if (window.performance && window.performance.mark){try {if (window.performance.getEntriesByName("NAVIGATE_START") && window.performance.getEntriesByName("NAVIGATE_START")[0]) { name = "CL_" + name ;}window.performance.mark(name);} catch (e) {console.warn(name + ' could not be marked:',e);}};};
window._perfMeasure = function _perfMeasure (name, start, end) {if (window.performance && window.performance.measure){try {if (window.performance.getEntriesByName("NAVIGATE_START") && window.performance.getEntriesByName("NAVIGATE_START")[0]) { start = "CL_" + start ;end = "CL_" + end ;name = "CL_" + name ;}window.performance.measure(name, start, end);} catch (e) {console.warn(name + ' could not be added:',e);}};};
window._pushAdPerfMetric = function _pushAdPerfMetric(key) {if (window.performance && window.performance.now) {_adPerfData.push([

In [70]:
CleanData_BS = list(filter(None,ls_BS))


In [71]:
CleanData_BS = CleanData_BS[10:]
CleanData_BS.remove('Quarterly')
CleanData_BS.remove('Expand All')
CleanData_BS.remove('Cash Flow')
CleanData_BS.remove('Balance Sheet')
CleanData_BS.insert(5,'Total Assets')
CleanData_BS.insert(10,'Total Liabilities')
CleanData_BS.insert(15,'Total Equity')


In [72]:
data_BS = list(zip(*[iter(CleanData_BS)]*5))
data_BS

[('Annual', '6/30/2020', '6/30/2019', '6/30/2018', '6/30/2017'),
 ('Total Assets', '301,311,000', '286,556,000', '258,848,000', '241,086,000'),
 ('Total Liabilities',
  '183,007,000',
  '184,226,000',
  '176,130,000',
  '168,692,000'),
 ('Total Equity', '118,304,000', '102,330,000', '82,718,000', '72,394,000'),
 ('Total Capitalization',
  '177,882,000',
  '168,992,000',
  '154,960,000',
  '148,467,000'),
 ('Common Stock Equity',
  '118,304,000',
  '102,330,000',
  '82,718,000',
  '72,394,000'),
 ('Capital Lease Obligations', '7,671,000', '6,188,000', '5,568,000', '-'),
 ('Net Tangible Assets',
  '67,915,000',
  '52,554,000',
  '38,982,000',
  '27,166,000'),
 ('Working Capital',
  '109,605,000',
  '106,132,000',
  '111,174,000',
  '95,324,000'),
 ('Invested Capital',
  '181,631,000',
  '174,508,000',
  '158,958,000',
  '158,588,000'),
 ('Tangible Book Value',
  '67,915,000',
  '52,554,000',
  '38,982,000',
  '27,166,000'),
 ('Total Debt', '70,998,000', '78,366,000', '81,808,000', '86,19

In [73]:
balance_sheet = pd.DataFrame(data_BS[:-1])
balance_sheet.columns = balance_sheet.iloc[0] # Name columns to first row of dataframe
balance_sheet = balance_sheet.iloc[1:,] # start to read 1st row
balance_sheet = balance_sheet.T # transpose dataframe
balance_sheet.columns = balance_sheet.iloc[0] #Name columns to first row of dataframe
balance_sheet.drop(balance_sheet.index[0],inplace=True) #Drop first index row
balance_sheet.index.name = '' # Remove the index name
balance_sheet.rename(index={'ttm': '12/31/%s' %date.today().year},inplace=True) #Rename ttm in index columns to end of the year
balance_sheet.rename(columns={'Annual':''},inplace=True)

In [85]:
print(balance_sheet)
print(balance_sheet.index)

Annual    Total Assets Total Liabilities  ... Share Issued Ordinary Shares Number
                                          ...                                    
6/30/2020  301,311,000       183,007,000  ...    7,571,000              7,571,000
6/30/2019  286,556,000       184,226,000  ...    7,643,000              7,643,000
6/30/2018  258,848,000       176,130,000  ...    7,677,000              7,677,000
6/30/2017  241,086,000       168,692,000  ...    7,708,000              7,708,000

[4 rows x 14 columns]
Index(['6/30/2020', '6/30/2019', '6/30/2018', '6/30/2017'], dtype='object', name='')


##### **Cash Flow Statement**

In [75]:
read_data_CF = ur.urlopen(url_CashFlowStmt).read() 
raw_CF = BeautifulSoup(read_data_CF,'lxml')

In [76]:
ls_CF = [] 
for data_CF in raw_CF.find_all('div'): 
  ls_CF.append(data_CF.string) 


In [77]:
CleanData_CF = list(filter(None,ls_CF))

In [78]:
CleanData_CF = CleanData_CF[13:]
CleanData_CF.remove('Quarterly')
CleanData_CF.remove('Expand All')
CleanData_CF.insert(6,'Operating Cash Flow')
CleanData_CF.insert(12,'Investing Cash Flow')
CleanData_CF.insert(18,'Financing Cash Flow')
CleanData_CF.insert(24,'End Cash Position')

In [79]:
data_CF = list(zip(*[iter(CleanData_CF)]*6))

In [80]:
cash_flow = pd.DataFrame(data_CF[:-1])
cash_flow.columns = cash_flow.iloc[0] # Name columns to first row of dataframe
cash_flow = cash_flow.iloc[1:,] # start to read 1st row
cash_flow = cash_flow.T # transpose dataframe
cash_flow.columns = cash_flow.iloc[0] #Name columns to first row of dataframe
cash_flow.drop(cash_flow.index[0],inplace=True) #Drop first index row
cash_flow.index.name = '' # Remove the index name
cash_flow.rename(index={'ttm': '12/31/%s' %date.today().year},inplace=True) #Rename ttm in index columns to end of the year
cash_flow.rename

<bound method DataFrame.rename of ttm        Operating Cash Flow  ...     -24,799,000
                                ...                
6/30/2020           60,675,000  ...     -22,968,000
6/30/2019           52,185,000  ...     -19,543,000
6/30/2018           43,884,000  ...     -10,721,000
6/30/2017           39,507,000  ...     -11,788,000
66,192,000         -15,818,000  ...  Free Cash Flow

[5 rows x 9 columns]>

In [81]:
cash_flow

ttm,Operating Cash Flow,Investing Cash Flow,Financing Cash Flow,End Cash Position,"-16,963,000","1,461,000",-,-,"-24,799,000"
,,,,,,,,,
6/30/2020,"60,675,000","-12,223,000","-46,031,000","13,576,000","-15,441,000","1,343,000",0,"-5,518,000","-22,968,000"
6/30/2019,"52,185,000","-15,773,000","-36,887,000","11,356,000","-13,925,000","1,142,000",0,"-4,000,000","-19,543,000"
6/30/2018,"43,884,000","-6,061,000","-33,590,000","11,946,000","-11,632,000","1,002,000","7,183,000","-10,060,000","-10,721,000"
6/30/2017,"39,507,000","-46,781,000","8,408,000","7,663,000","-8,129,000","772,000","44,344,000","-7,922,000","-11,788,000"
"66,192,000","-15,818,000","-46,111,000","17,380,000",Capital Expenditure,Issuance of Capital Stock,Issuance of Debt,Repayment of Debt,Repurchase of Capital Stock,Free Cash Flow
